In [3]:
import numpy as np
import pandas as pd

In [10]:
points = pd.read_csv('data/TSP.csv', header=None, index_col=0, names=["x", "y"])
points.head()

,x,y
1,565,575
2,25,185
3,345,750
4,945,685
5,845,655


# Random path

In [ ]:
def get_random_path():
    